In [ ]:
from transformers import pipeline
# class for tokenizer and sequence classification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from pathlib import Path

In [9]:
# simple pretrained classifier
classifier = pipeline("sentiment-analysis")
res = classifier("How are you doing today?")
print(res)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9429818391799927}]


You can find different models [here]("")

from_pretrained(model_name): load pretrained model for any architecture

In [10]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
res = classifier("How are you doing today?")

observe that input_ids includes 101 and 102, which indicate the beginning and end respectively

In [11]:
tokens = tokenizer.tokenize("We are very happy to show you the huggingface transformers library")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer("We are very happy to show you the huggingface transformers library")

print("Tokens: %s" % tokens)
print("Token IDs: %s" % token_ids)
print("Input IDs: %s" % input_ids)


Tokens: ['we', 'are', 'very', 'happy', 'to', 'show', 'you', 'the', 'hugging', '##face', 'transformers', 'library']
Token IDs: [2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 17662, 12172, 19081, 3075]
Input IDs: {'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 17662, 12172, 19081, 3075, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Pad or truncate all sample points for consistency

In [12]:
X_train = ["We are very happy to show you the HuggingFace Transformers library",
            "We hope you don't hate it."]

batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt")

automodel lets you add labels=torch.tensor([1,0])
which outputs loss for SequenceClassifierOutput

Below is the manual implementation

In [13]:
with torch.no_grad():
    # batch is a dictionary, so we unpack them
    outputs = model(**batch, labels=torch.tensor([1, 0]))
    print(outputs)
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=tensor(0.3168), logits=tensor([[-4.0383,  4.2478],
        [ 0.0818, -0.0418]]), hidden_states=None, attentions=None)
tensor([[2.5194e-04, 9.9975e-01],
        [5.3086e-01, 4.6914e-01]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


# Finetuning the Model
save your finetuned model:

In [14]:
save_directory = "saved"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

loading in a pretrained model:

In [15]:
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory)

# Suppose we want to classify texts in German:

In [16]:
model_name = "oliverguhr/german-sentiment-bert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

X_train = ["", "", ""]

batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, returns_tensors="pt")

with torch.no_grad():
    outputs = model(**batch)
    label_ids = torch.argmax(outputs.logits, dim=1)
    print(label_ids)
    labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Finetuning our own models
[helpful link!](https://huggingface.co/transformers/v3.2.0/custom_datasets.html)

### general approach: 
1. prepare dataset
2. load pretrained tokenizer, call it with dataset -> encoding
3. build pytorch dataset with encodings
4. load pretrained model
5. 
    a) load trainer and train it
    b)  or use native pytorch training pipeline

In [ ]:
model_name = "distilbert-base-uncased"

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == "neg" else 1)